In [1]:
import os, sys
dir1 = os.path.dirname(os.path.abspath(''))
if not dir1 in sys.path: sys.path.append(dir1)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.style.use('seaborn-v0_8-darkgrid')
import pandas as pd
import numpy as np
from evaluation import calculation
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as co
from IPython.display import display
pd.options.mode.chained_assignment = None
import pandas as pd
import pickle
import feature.feature_lag_selection as fls
import feature.feature_external_selection as fes
import feature.feature_transformation as ft
import feature.time_categorical as tc
import feature.time_stationarization as ts
import models.model_init as mi
import models.benchmark_init as bi
import evaluation.metrics as em
import postprocessing.quantile as ppq
import postprocessing.value as ppv
import datetime as dt
from forecast.scenario import calculate_scenario
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import warnings
warnings.filterwarnings("ignore")

In [ ]:
site_id = "GEF14"
file_name = "load_with_weather.pkl"
data = pd.read_pickle(f"../data/{site_id}/{file_name}")[17000:]

target = ["load"]
quantiles = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
methods_to_train = [  
                      
                      mi.MQKNNR(quantiles),
                      mi.MQRFR(quantiles),
                      mi.MQERT(quantiles),
                     mi.MQDLinear(quantiles,device,'ex_Linear'),
                   mi.MQLSTM(quantiles,device,'ex_Linear'),
                    mi.MQMLP(quantiles,device,'ex_Linear'),
                   mi.MQCNN(quantiles,device,'ex_Linear'),
                   mi.MQTransformer(quantiles,device,'ex_Linear'),
                      mi.MQLSTNet(quantiles,device,'ex_Linear'),
                    mi.MQInformer(quantiles,device,'ex_Linear'),
                    mi.MQAutoformer(quantiles,device,'ex_Linear'),
                    mi.MQFedformer(quantiles,device,'ex_Linear'),
                    mi.MQFiLM(quantiles,device,'ex_Linear'),
                    mi.MQiTransformer(quantiles,device,'ex_Linear'),
                    mi.MQNSTransformer(quantiles,device,'ex_Linear'),
                      mi.MQSegRNN(quantiles,device,'ex_Linear'),
                      mi.MQTimeMixer(quantiles,device,'ex_Linear'),
                      mi.MQTimesNet(quantiles,device,'ex_Linear'),
                        mi.MQTSMixer(quantiles,device,'ex_Linear'),
                      mi.MQFreTS(quantiles,device,'ex_Linear'),
                      mi.MQReformer(quantiles,device,'ex_Linear'),
                  mi.MQNBEATS(quantiles,device,'ex_Linear'),
                    mi.MQWaveNet(quantiles,device,'ex_Linear'),
                    mi.MQMICN(quantiles,device,'ex_Linear'),
                    mi.MQTimeXer(quantiles,device,'ex_Linear'),
                  mi.MQNBEATSX(quantiles,device),
                  mi.MQTSMixerExt(quantiles,device),
                   mi.MQBiTCN(quantiles,device),
                   mi.MQTiDE(quantiles,device),
                    mi.MQTFT(quantiles,device),
                    ]
horizon = 1  # or int(dt_resolution.seconds/data.index.freq.n)
train_ratio = 0.8
lags_list = list(range(1, 25))
preds_list = list(range(1, 24))
feature_transformation = ft.NoTransformationStrategy()
time_stationarization = ts.NoStationarizationStrategy()
datetime_features = [tc.Hour(),tc.Day(),tc.Weekday(),tc.Month()]
target_lag_selection = fls.ManualStrategy(lags=lags_list)
target_pred_selection = fls.ManualStrategy(lags=preds_list)
external_feature_selection = fes.LagStrategy({"airTemperature":fls.ManualStrategy(lags=lags_list)},
                                             {"airTemperature":fls.ManualStrategy(lags=preds_list)})
evaluation_metrics = [
                      em.ReliabilityMatrix(quantiles),
                      em.CalibrationMatrix(quantiles),
                      em.WinklerScoreMatrix(quantiles),
                      em.IntervalWidthMatrix(quantiles),
                      em.RampScoreMatrix(threshold=0.1, normalize=True),
                      em.PinballLossMatrix(quantiles),
                      em.QuantileCrossingMatrix(quantiles),
                      em.PinballLoss(quantiles),
                     #  em.CRPSError(quantiles),
                      em.CalibrationError(quantiles),
                      em.BoundaryCrossing(0, None),
                      em.QuantileCrossing(quantiles),
                      em.Skewness(),
                      em.Kurtosis(),
                      em.QuartileDispersion()
                      ]
post_processing_quantile = ppq.QuantileSortingStrategy()
post_processing_value = ppv.ValueClippingStrategy(0, None)
save_results = False

err_tot, forecast_tot,true_tot = calculate_scenario(data=data,
                                            target=target,
                                            methods_to_train=methods_to_train,
                                            horizon=horizon,
                                            train_ratio=train_ratio,
                                            feature_transformation=feature_transformation,
                                            time_stationarization=time_stationarization,
                                            datetime_features=datetime_features,
                                            target_lag_selection=target_lag_selection,
                                            target_pred_selection=target_pred_selection,
                                            external_feature_selection=external_feature_selection,
                                            post_processing_quantile=post_processing_quantile,
                                            post_processing_value=post_processing_value,
                                            evaluation_metrics=evaluation_metrics,
                                            device=device)